<a href="https://colab.research.google.com/github/lugsantistebanji/WCS-IA/blob/main/WCS_IA_Quetes_API_GeoCoding_with_Nominatim_OSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
The Nominatim API is a REST API (provided by OpenStreetMap), that returns geographic coordinates (latitude and longitude) from a postal address. If several postal addresses (due to inaccuracy or non-existent street number) corresponding to the request are found, the API returns several coordinates, each time with an `importance` score. The coordinates are given in descending order of importance, so you can select only the first address.

In [3]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.0 MB/s eta 0:00:00


In [2]:
# Execute the code below :
import requests
from unidecode import unidecode


link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0',
    'Referer': 'https://www.example.com'
}
r = requests.get(link, headers=headers).json()
r

[{'place_id': 48174245,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 1125079468,
  'lat': '37.225196722061376',
  'lon': '14.52468563676896',
  'class': 'highway',
  'type': 'residential',
  'place_rank': 26,
  'importance': 0.05339371862363619,
  'addresstype': 'road',
  'name': 'Via Pietro Mascagni',
  'display_name': 'Via Pietro Mascagni, Caltagirone, Catania, Sicilia, 95041, Italia',
  'boundingbox': ['37.2250444', '37.2253490', '14.5244318', '14.5249395']},
 {'place_id': 48194427,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 58232256,
  'lat': '37.6091344949263',
  'lon': '15.159672789802412',
  'class': 'highway',
  'type': 'residential',
  'place_rank': 26,
  'importance': 0.0533936133363851,
  'addresstype': 'road',
  'name': 'Via Pietro Mascagni',
  'display_name': 'Via Pietro Mascagni, Sciarelle, Acireale, Catania, Sicilia, 95024,

In [16]:
# Here we select only the first address (index 0)
print("First address :", r[0])
print("First address longitude :",r[0]['lon'])
print("First address latitude :",r[0]['lat'])

First address : {'place_id': 48174245, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 1125079468, 'lat': '37.225196722061376', 'lon': '14.52468563676896', 'class': 'highway', 'type': 'residential', 'place_rank': 26, 'importance': 0.05339371862363619, 'addresstype': 'road', 'name': 'Via Pietro Mascagni', 'display_name': 'Via Pietro Mascagni, Caltagirone, Catania, Sicilia, 95041, Italia', 'boundingbox': ['37.2250444', '37.2253490', '14.5244318', '14.5249395']}
First address longitude : 14.52468563676896
First address latitude : 37.225196722061376


In [13]:
# For ease of use, you can add a limit on the number of items returned:

link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json"
r = requests.get(link, headers=headers).json()
print("WITHOUT limit, how many coordinates does this address return?",len(r))

link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json&limit=1"
r = requests.get(link, headers=headers).json()
print("WITH limit, how many coordinates does this address return?",len(r))

WITHOUT limit, how many coordinates does this address return? 10
WITH limit, how many coordinates does this address return? 1


## How to create your API query
It's up to you to modify the string to create the right request URL

In [18]:
# We observe that the query consists of a fixed part, followed by the address to be searched for.
# An URL cannot contain a " " space character,
# and special characters or accents should be avoided if possible

link_main = 'https://nominatim.openstreetmap.org/?q='
address = '54 Via Pietro Mascagni, Catania, Italy'
link_end = '&format=json&limit=1'

link = link_main + address.replace(", ",",").replace(" ", "+")
print(link + link_end)

https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json&limit=1


In [40]:
def get_url_with_address_normalized(adresse_postale):
    link_main = 'https://nominatim.openstreetmap.org/?q='
    adress = adresse_postale.replace(", ",",").replace(" ", "+")
    link_end = '&format=json&limit=1'
    link = link_main + adress + link_end
    return link

In [44]:
# Create a function here that turns a postal address into a request URL for the Nominatim API,
# then makes the request and returns the coordinates :

def API_address(adresse_postale):
    coordinates = []
    link = get_url_with_address_normalized(adresse_postale)
    response = requests.get(link, headers=headers)

    if response.status_code == 200:
        data = response.json()
        coordinates = (data[0]['lat'], data[0]['lon'])
    else:
        print(f"Request error : {response.status_code}")

    return tuple(coordinates)

In [47]:
# Test it here:
API_address(address)

('37.225196722061376', '14.52468563676896')

# DataViz
Latitude & Longitude can be used on visualization tools, whether they are BI tools (PowerBI, Table), or Python DataViz libraries such as Plotly or Folium.

Here we will display a map with Folium.


In [22]:
# The syntax of Folium is very simple, you start by creating a map centered on a point.
# You can change the default zoom level with the argument "zoom_start".

import folium

# We define a location as a list with 2 values : latitude and longitude.
point = [float(r[0]['lat']), float(r[0]['lon'])]

# We center the map on the location
m = folium.Map(location=point,zoom_start=7)

# We display the map
m

In [23]:
# Then you can add landmarks and put a clickable comment
m = folium.Map(location=point, )
folium.Marker(
    location=point,
    popup='a good restaurant'
    ).add_to(m)
m

# Challenge
Here is a DataFrame with restaurants in Catania, Sicily, and their respective addresses. Here is your mission:
- Create a new column "coordinates", which will store the coordinates corresponding to each address (you can use the function you created previously).
- Display a map with the 4 restaurant markers. Be careful, the restaurants are very close, remember to set the default zoom level so that it is clearly legible. You can center the map on the first restaurant. And display the name of the restaurant in the tooltip popup.

1. __DataFrame__

In [24]:
import pandas as pd
restaurants = pd.DataFrame([["Gelateria Zio Pietro dal 1964", "Via Porta di Ferro, 47, 95131 Catania CT"],
                            ["A Casa d'Amici","Via Fischetti, 14, 95131 Catania CT"],
                            ["La Bitta", "Via Acquicella Porto, 95121 Catania CT"],
                            ["Steak House", "Via Porta di Ferro, 8, 95100 Catania CT"]
                            ],
                           columns = ["name", "address"])

restaurants

,name,address
0,Gelateria Zio Pietro dal 1964,"Via Porta di Ferro, 47, 95131 Catania CT"
1,A Casa d'Amici,"Via Fischetti, 14, 95131 Catania CT"
2,La Bitta,"Via Acquicella Porto, 95121 Catania CT"
3,Steak House,"Via Porta di Ferro, 8, 95100 Catania CT"


2. __Fetching coordinates__

In [50]:
restaurants['coordinates'] = restaurants['address'].apply(API_address)

In [51]:
restaurants

,name,address,coordinates
0,Gelateria Zio Pietro dal 1964,"Via Porta di Ferro, 47, 95131 Catania CT","(37.5020931, 15.0931951)"
1,A Casa d'Amici,"Via Fischetti, 14, 95131 Catania CT","(37.506171, 15.0943223)"
2,La Bitta,"Via Acquicella Porto, 95121 Catania CT","(37.48944028886065, 15.079431220911957)"
3,Steak House,"Via Porta di Ferro, 8, 95100 Catania CT","(37.5029335, 15.0930406)"


3. __Display map__

In [55]:
import folium

mapp = folium.Map(location=restaurants.loc[0,'coordinates'], zoom_start=14)

for index, restaurant in restaurants.iterrows():
    folium.Marker(
    location=restaurant['coordinates'],
    popup=f"{restaurant['name']} - \n{restaurant['address']} - \n{restaurant['coordinates']}"
    ).add_to(mapp)
mapp

# Remarks on the Nominatim API
As indicated in the quest, there are many resources for geocoding. Most of them are available by registration, and some require a fee.

The Nominatim API is free and without registration. The disadvantage is that it is relatively slow. If you need to use it in the future, remember to store the results so you don't have to run it several times.

For your knowledge, there is also :
- the **reverse** address API, which allows you to find the nearest postal address using geographic coordinates.
- the API from a **CSV file** if you have a lot of addresses to geocode
- the API **GeoJSON** which allows to obtain a geoJSON format of locations to make choropleth maps


All the [documentation is available here](https://nominatim.org/release-docs/develop/api/Search/).